In [17]:
import numpy as np
import pandas as pd
import glob
import math

import nibabel as nib
from nibabel.filebasedimages import FileBasedImage

import matplotlib.pyplot as plt
import cv2

from skimage import measure
from skimage import morphology
from scipy import ndimage as ndi
from scipy.spatial import ConvexHull
from scipy.ndimage import binary_fill_holes
from skimage.feature import peak_local_max
from skimage.segmentation import watershed

from surface_distance.metrics import compute_surface_distances, compute_robust_hausdorff, compute_dice_coefficient

In [18]:
def read_nii(nii_path):
    nii = nib.load(nii_path)
    nii_data = nii.get_fdata()
    return nii_data

def dice_coeff(lung, lung_ref):
    # calculate the dice coefficient
    dice = compute_dice_coefficient(lung, lung_ref)
    return dice

def hausdorff_dist(lung, lung_ref):
    # calculate the directed hausdorff distance in 3d
    distance = compute_surface_distances(lung, lung_ref, spacing_mm=(1, 1, 1))
    hausdorff = compute_robust_hausdorff(distance, 95)
    return hausdorff

In [19]:
fast_segmentation_imgs = glob.glob('./data/fast/*.nii.gz')
fast_segmentation_imgs.sort()

normal_segmentation_imgs = glob.glob('./data/normal/*.nii.gz')
normal_segmentation_imgs.sort()

reference_imgs = glob.glob('../../lab-1/Lab1/ReferenceSegmentations/*.nii.gz')
reference_imgs.sort()

our_own_segmentation_imgs = glob.glob('./data/lung_segmentation_output/*.nii.gz')
our_own_segmentation_imgs.sort()

total_segmenter_fast_dice = []
total_segmenter_normal_dice = []
total_segmenter_normal_to_our_algo = []

for i in range(len(fast_segmentation_imgs)):
    fast_segmentation = read_nii(fast_segmentation_imgs[i])
    normal_segmentation = read_nii(normal_segmentation_imgs[i])
    reference = read_nii(reference_imgs[i])
    our_own_segmentation_img = read_nii(our_own_segmentation_imgs[i])

    # Convert to binary
    fast_segmentation = (fast_segmentation > 0).astype(np.bool_)
    normal_segmentation = (normal_segmentation > 0).astype(np.bool_)
    reference = (reference > 0).astype(np.bool_)
    our_own_segmentation = (our_own_segmentation_img > 0).astype(np.bool_)

    dice_fast = dice_coeff(fast_segmentation, reference)
    total_segmenter_fast_dice.append(dice_fast)
    hausdorff_fast = hausdorff_dist(fast_segmentation, reference)

    dice_normal = dice_coeff(normal_segmentation, reference)
    total_segmenter_normal_dice.append(dice_normal)
    hausdorff_normal = hausdorff_dist(normal_segmentation, reference)

    dice_our_algo = dice_coeff(normal_segmentation, our_own_segmentation)
    total_segmenter_normal_to_our_algo.append(dice_our_algo)
    hausdorff_our_algo = hausdorff_dist(normal_segmentation, our_own_segmentation)

    print(f"Fast segmentation: Dice: {dice_fast}, Hausdorff: {hausdorff_fast}")
    print(f"Normal segmentation: Dice: {dice_normal}, Hausdorff: {hausdorff_normal}")
    print(f"Our own segmentation: Dice: {dice_our_algo}, Hausdorff: {hausdorff_our_algo}")
    print()

print(f"Average dice coefficient for fast segmentation: {np.mean(total_segmenter_fast_dice)}")
print(f"Average dice coefficient for normal segmentation: {np.mean(total_segmenter_normal_dice)}")
print(f"Average dice coefficient for normal segmentation to our own algo: {np.mean(total_segmenter_normal_to_our_algo)}")



Fast segmentation: Dice: 0.9362954463246633, Hausdorff: 10.295630140987
Normal segmentation: Dice: 0.9470788380804278, Hausdorff: 11.180339887498949
Our own segmentation: Dice: 0.9481736294553837, Hausdorff: 9.486832980505138

Fast segmentation: Dice: 0.936084867905894, Hausdorff: 14.035668847618199
Normal segmentation: Dice: 0.9608472162823355, Hausdorff: 11.575836902790225
Our own segmentation: Dice: 0.9578565653791681, Hausdorff: 13.0

Fast segmentation: Dice: 0.9656273786433148, Hausdorff: 7.3484692283495345
Normal segmentation: Dice: 0.9767830755470718, Hausdorff: 10.198039027185569
Our own segmentation: Dice: 0.9786829084028822, Hausdorff: 6.082762530298219

Fast segmentation: Dice: 0.9270037214252702, Hausdorff: 15.0
Normal segmentation: Dice: 0.9303787911320011, Hausdorff: 16.64331697709324
Our own segmentation: Dice: 0.7930838682589463, Hausdorff: 53.81449618829484

Fast segmentation: Dice: 0.9239395630192143, Hausdorff: 10.770329614269007
Normal segmentation: Dice: 0.94031948